# BinXray Patch Detection on BCSD Outputs

In [17]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from glob import glob
import sys,os
from pathlib import Path
import csv
import json
from collections import defaultdict

In [2]:
os.chdir("/home/angr/BinXray/")

## BinXray detection interface

In [3]:
sys.path.append("/home/angr/BinXray/")
from detection_main import sim_cal_for_bcsd

In [ ]:
sim_cal_for_bcsd("CVE-2014-0160", "binaries/openssl/O2/openssl-1.0.1m", "dtls1_process_heartbeat", "dtls1_process_heartbeat", False)

In [5]:
OPENSSLM_VUL_IN_CVE = {"CVE-2015-0293", "CVE-2015-1788", "CVE-2015-1790", "CVE-2015-1789", "CVE-2015-1792",
                       "CVE-2015-1791", "CVE-2016-2108", "CVE-2015-1793", "CVE-2015-3196", "CVE-2015-3195",
                       "CVE-2015-3194", "CVE-2015-3197", "CVE-2016-0797", "CVE-2016-0702", "CVE-2016-0705",
                       "CVE-2016-0798", "CVE-2016-2105", "CVE-2016-2106", "CVE-2016-2176", "CVE-2016-2107",
                       "CVE-2016-2109", "CVE-2016-2182", "CVE-2016-6303", "CVE-2016-2180", "CVE-2016-2178",
                       "CVE-2016-6306", "CVE-2016-2181", "CVE-2016-2179", "CVE-2016-2177", "CVE-2016-6304",
                       "CVE-2016-6302"}
OPENSSL_101M_CVE_MARK = list(csv.reader(open('/home/angr/PatchDiff/data/SAFE/openssl_patch', 'r')))
# convert OPENSSL_101M_CVE_MARK to dic
openssl_cve_dic = {}
for cve, func_name, mark in OPENSSL_101M_CVE_MARK:
    openssl_cve_dic[cve] = (func_name, mark)

In [87]:
TARGET_BIN = '/home/angr/PatchDiff/binaries/openssl/O0/openssl-1.0.1m'

# Gemini

## Load BCSD results

In [7]:
GEMINI_result_dir = "/home/angr/PatchDiff/data/Gemini/func_match_res/"
detection_res = []

## Detecting Patch

In [ ]:
visited = set()
for cal_res in detection_res:
    visited.add((cal_res[0], cal_res[1]))

for res_file in tqdm(glob(GEMINI_result_dir + "CVE*")):
    sp = res_file.split("+")
    if len(sp) ==1:
        continue

    vul_func_name = sp[1][: sp[1].rfind('_')]
    cveid = sp[0].split("/")[-1]
    
    
    
    # load res
    res_list = json.load(open(res_file, 'r'))

    # deduplication
    pre = (0, 0)
    new_list = []
    for item in res_list:
        if item != pre:
            new_list.append(item)
            pre = item

    top50 = new_list[:50]

    # # get all candidate functions
    candidate_functions = [x[1] for x in top50]

    #    :param mark: V|P|U; vulnerable, patched, unknown
    for detect_name in candidate_functions:
        
        if (cveid, detect_name) in visited:
            continue
        try:
            s = sim_cal_for_bcsd(cveid, TARGET_BIN, vul_func_name, detect_name, False)
            mark = "U"  # vulnerability-unrelated functions
            if vul_func_name == detect_name:
                if cveid not in openssl_cve_dic:
                    continue
                mark = openssl_cve_dic[cveid][1]
            detection_res.append((cveid, detect_name, mark, s))
        except EOFError:
            pass
   

In [14]:
len(detection_res)

3141

## FPR 

In [ ]:
detection_res_dic

* Calculating the function rankings

 **For the results with "NA too much diff" or "NT no trace", we set the scores to 0.**

In [41]:
def cal_patch_ranking(detection_res):
    detection_res_dic = defaultdict(list)
    for res in detection_res:
        detection_res_dic[res[0]].append(res[1:])
    patch_func_rank = {}
    for cve in detection_res_dic:
        if cve in openssl_cve_dic and openssl_cve_dic[cve][1] == 'P':
            cve_res = detection_res_dic[cve]
            # convert the output of binxray to score
            for func_res in cve_res:
                res = func_res[2]
                if res == "NA too much diff":
                    score = 0
                elif res[0]=="P":
                    if len(res) == 1:
                        score = 1
                    else:
                        score= float((res.split(" ")[1]).split("/")[0])
                elif res[0] == "V":
                    if res == "V no main changes" or len(res) == 1:
                        score = -1.0
                    else:
                        score= -1.0*float((res.split(" ")[1]).split("/")[0])
                else:
                    score = 0
            # calculate the ranking of patched function
            sorted_res = sorted(cve_res, key = lambda x:x[2], reverse=True)
            rank = 50
            for i, x in enumerate(sorted_res):
                if x[1] == "P":
                    rank = i
                    break
            patch_func_rank[cve] = rank
    return patch_func_rank, detection_res_dic

In [43]:
ranks, res_dic = cal_patch_ranking(detection_res)

In [44]:
ranks

{'CVE-2010-5298': 0,
 'CVE-2014-3569': 0,
 'CVE-2015-0204': 0,
 'CVE-2014-3507': 0,
 'CVE-2015-0292': 0,
 'CVE-2014-8176': 0,
 'CVE-2014-3572': 0,
 'CVE-2014-3510': 0,
 'CVE-2014-0221': 6,
 'CVE-2013-6450': 50,
 'CVE-2014-3505': 0,
 'CVE-2013-4353': 0,
 'CVE-2015-0206': 28,
 'CVE-2014-3512': 50,
 'CVE-2014-3567': 2,
 'CVE-2014-3511': 0,
 'CVE-2014-3470': 0,
 'CVE-2014-3513': 6,
 'CVE-2014-0224': 0,
 'CVE-2014-0160': 3,
 'CVE-2014-0198': 0,
 'CVE-2014-8275': 1,
 'CVE-2015-0205': 0,
 'CVE-2013-0166': 17,
 'CVE-2014-3508': 3,
 'CVE-2013-6449': 4}

In [48]:
res_dic["CVE-2010-5298"]

[('ssl3_read_bytes', 'P', 'NA too much diff'),
 ('dtls1_read_bytes', 'U', 'NA too much diff'),
 ('ec_wNAF_mul', 'U', 'NA too much diff'),
 ('do_body', 'U', 'NA too much diff'),
 ('ssl_parse_serverhello_tlsext', 'U', 'NA too much diff'),
 ('ssl2_accept', 'U', 'NA too much diff'),
 ('b64_read', 'U', 'NA too much diff'),
 ('ts_main', 'U', 'NA too much diff'),
 ('X509_NAME_oneline', 'U', 'NA too much diff'),
 ('sub_80AF5FC', 'U', 'NA too much diff'),
 ('OCSP_sendreq_nbio', 'U', 'NA too much diff'),
 ('sv_body', 'U', 'NA too much diff'),
 ('engine_main', 'U', 'NA too much diff'),
 ('d2i_SSL_SESSION', 'U', 'NA too much diff'),
 ('parseArgs', 'U', 'NA too much diff'),
 ('SSL_SESSION_print', 'U', 'NA too much diff'),
 ('ecparam_main', 'U', 'NA too much diff'),
 ('ssl23_get_client_hello', 'U', 'NA too much diff'),
 ('do_name_ex', 'U', 'NA too much diff'),
 ('enc_main', 'U', 'NA too much diff'),
 ('OBJ_obj2txt', 'U', 'NA too much diff'),
 ('EVP_BytesToKey', 'U', 'NA too much diff'),
 ('dtls1_do_

In [80]:
def FPR_AT_TopN(n, ranks):
    Top_N=n
    num_fp = 0
    for cve, rank in ranks.items():
        if rank<Top_N:
            num_fp +=1
    return num_fp/len(ranks)

In [81]:
FPR_AT_TopN(1, ranks),FPR_AT_TopN(5, ranks),FPR_AT_TopN(10, ranks),FPR_AT_TopN(20, ranks)

(0.5769230769230769,
 0.7692307692307693,
 0.8461538461538461,
 0.8846153846153846)

# Bingo

## Load results

In [57]:
Bingo_RES_DIR = '/home/angr/bingo/results'
detection_res_for_bingo = []
map_dic = {}
with open('/home/angr/bingo/cve_func_bin.txt', 'r') as f:
    for l in f.readlines():
        l = l.strip()
        cve, func, bin = l.split(" ")
        map_dic[(func, bin)] = cve

## Detecting Patch

In [ ]:
visited = set()
for cal_res in detection_res_for_bingo:
    visited.add((cal_res[0], cal_res[1]))

for res_file in tqdm(glob(Bingo_RES_DIR + "/*.bak.test")):
    res_file = os.path.basename(res_file)
    print(res_file)
    sp = res_file.split("+")
    ssp = sp[1].split("_vs_")
    vul_func_name = ssp[0]
    ssp[1] = ssp[1].replace('.bak','')
    # TARGET_BIN = os.path.join("/home/angr/PatchDiff/binaries/openssl/O0", ssp[1][:-5])
    bin = sp[0]
    if (vul_func_name, bin) not in map_dic:
        continue
    cveid = map_dic[(vul_func_name, bin)]
    # load res
    with open(os.path.join(Bingo_RES_DIR, res_file), 'r') as f:
        top50_funcs = f.readlines()[1:51]

    for func_score in top50_funcs:
        detect_name = func_score.strip().split(":")[0]
        if (cveid, detect_name) in visited:
            continue
        try:
            s = sim_cal_for_bcsd(cveid, TARGET_BIN, vul_func_name, detect_name, False)
            mark = "U"  # 不是漏洞函数也不是补丁函数，无关函数
            if vul_func_name == detect_name:
                if cveid not in openssl_cve_dic:
                    continue
                mark = openssl_cve_dic[cveid][1]
            detection_res_for_bingo.append((cveid, detect_name, mark, s))
        except EOFError:
            pass

In [ ]:
detection_res_for_bingo[0:10]

## FPR

In [91]:
ranks_for_bingo, res_dic_for_bingo = cal_patch_ranking(detection_res_for_bingo)

In [92]:
FPR_AT_TopN(1, ranks_for_bingo),FPR_AT_TopN(5, ranks_for_bingo),FPR_AT_TopN(10, ranks_for_bingo),FPR_AT_TopN(20, ranks_for_bingo)

(0.5416666666666666,
 0.7916666666666666,
 0.8333333333333334,
 0.9166666666666666)

# SAFE

## Load Results

In [67]:
SAFE_RES_DIR = '/home/angr/BinXray/binaries/openssl/O0/res-0427'
detection_res_for_safe = []

## Detecting Patch

In [ ]:
visited = set()
for cal_res in detection_res_for_safe:
    visited.add((cal_res[0], cal_res[1]))

for res_file in tqdm(glob(SAFE_RES_DIR + "/*.res")):
    
    if "CVE-2014-0160" in res_file or "CVE-2015-1789" in res_file: # execution stucks for this
        continue
        
    sp = os.path.basename(res_file).split("+")
    print("Detecting patch for "+ res_file)
    # vul_func_name = sp[1].replace("_openssl101m.res", "")
    vul_func_name = sp[1][: sp[1].rfind('_')]
    cveid = sp[0]

    # load res
    res_list = json.load(open(os.path.join(SAFE_RES_DIR, res_file), 'r'))

    # deduplication
    pre = (0, 0)
    new_list = []
    for item in res_list:
        if item != pre:
            new_list.append(item)
            pre = item

    top50 = new_list[:50]

    # # get all candidate functions
    candidate_functions = [x[0] for x in top50]


    #    :param mark: V|P|U; vulnerable, patched, unknown
    for detect_name in candidate_functions:
        if (cveid, detect_name) in visited:
            continue
        try:
            s = sim_cal_for_bcsd(cveid, TARGET_BIN, vul_func_name, detect_name, False)
            mark = "U"  # 不是漏洞函数也不是补丁函数，无关函数
            if vul_func_name == detect_name:
                if cveid not in openssl_cve_dic:
                    continue
                mark = openssl_cve_dic[cveid][1]
            detection_res_for_safe.append((cveid, detect_name, mark, s))
        except EOFError:
            pass

In [112]:
df_safe_res = pd.DataFrame(detection_res_for_safe, columns=["CVE", "Function", "Label", "BinXray Output"])

In [115]:
df_safe_res.sample(20)

,CVE,Function,Label,BinXray Output
1001,CVE-2010-5298,check_policy,U,NA too much diff
1777,CVE-2016-6304,CRYPTO_ofb128_encrypt,U,NA too much diff
1812,CVE-2014-0195,BN_nist_mod_384,U,NA too much diff
853,CVE-2014-3508,ssl23_get_server_hello,U,NA too much diff
2412,CVE-2015-0204,asn1_item_print_ctx,U,NA too much diff
1500,CVE-2016-6306,ssl_parse_clienthello_renegotiate_ext,U,C can't tell
841,CVE-2016-2182,hash_step,U,NT no trace
952,CVE-2014-3513,gost_imit_final,U,V 0.3667711598746082/0.2818029372319928 11/11
437,CVE-2014-3569,ssl_get_prev_session,U,NA too much diff
1991,CVE-2016-2176,b64_read,U,NA too much diff


## FPR

In [116]:
ranks_for_safe, res_dic_for_safe = cal_patch_ranking(detection_res_for_safe)

In [117]:
FPR_AT_TopN(1, ranks_for_safe),FPR_AT_TopN(5, ranks_for_safe),FPR_AT_TopN(10, ranks_for_safe),FPR_AT_TopN(20, ranks_for_safe)

(0.46153846153846156,
 0.6538461538461539,
 0.7692307692307693,
 0.8076923076923077)